# Epic Seven Gear Simulator Testing Documentation
This notebook documents the testing process.

* The json data files are stored in the `/data` folder. The scripts to prepare these json file are also in the `/data` folder.
* The test modules (along with this notebook) are stored in the `/tests` folder.
* The class and function modules for the app are stored in the `/src` folder.
* `unittest` module is used for running the tests.

### Directory Structure Issue

Modules in `/tests` folder have to import data and modules from the `/data` or `/src` folders respectively, so we need to add the parent directory to the path.

This is solved by importing `set_directory` function from the `set_directory_function` module. As such, every testing module should have the following two lines of code at the top, so that the parent folder can be added to the path.

In [ ]:
from set_directory_function import set_directory
set_directory()

### Running the tests
The tests must be run from the root directory as:

python tests/test_module_name.py

e.g.,
python tests/test_get_stat_by_id.py

## Testing Stat Class

### get_stat_by_id()

Test Module: [test_get_stat_by_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_stat_by_id.py)

The Stat() class initiates with no data. Running the get_stat_by_id() method should return a dictionary from the STATS dictionary based on the id provided. The function should be able to handle both int and str arguments as long as the number is between 0 and 10. So we need to test the following cases.

Valid stat_id should return a dictionary containing stats where id is 0 or 7 (compare to STATS[0] and STATS[7]):
* stat_id = 0 
* stat_id = '0' 
* stat_id = 7 
* stat_id = '7' 

Invalid stat_id should return None:
* stat_id = 100
* stat_id = '100'
* stat_id = -1 (negative case)
* stat_id = 1000000 (big number)

Result: All tests passed